<a href="https://colab.research.google.com/github/NNL-Keerthana/DeepLearning_EEG/blob/main/TSception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
dataset_path = '/content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0'
os.chdir(dataset_path)
#/content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/subject-info.csv

In [ ]:
!pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 22.7 MB/s eta 0:00:00


In [ ]:
import mne
import numpy as np
import pandas as pd

# Load subject info from subject-info.csv
subject_info = pd.read_csv('/content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/subject-info.csv')

# Function to calculate band-wise PSD
def calculate_band_psd(raw, sfreq, freq_bands):
    psds, freqs = mne.time_frequency.psd_array_multitaper(raw.get_data(), sfreq=sfreq, fmin=0.5, fmax=40, adaptive=True, normalization='full', verbose=0)
    band_psd = dict()
    for band, (fmin, fmax) in freq_bands.items():
        band_psd[band] = np.mean(psds[:, (freqs >= fmin) & (freqs < fmax)], axis=1)
    return band_psd

# Define frequency bands of interest
freq_bands = {
    'Delta': (1, 4),
    'Theta': (4, 8),
    'Alpha': (8, 12),
    'Beta': (12, 30),
    'Gamma': (30, 40)
}

# Function to load and preprocess data
def load_and_save_eeg_data(subject_info, freq_bands):
    X_list = []
    y_list = []
    for idx, row in subject_info.iterrows():
        subject_id = row['Subject']  # Directly use 'Subject' column from your CSV
        # File paths
        file_rest = f'/content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/{subject_id}_1.edf'
        file_task = f'/content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/{subject_id}_2.edf'

        # Load EDF files
        raw_rest = mne.io.read_raw_edf(file_rest, preload=True)
        raw_task = mne.io.read_raw_edf(file_task, preload=True)

        # Filter raw data
        raw_rest.filter(1, 40, fir_design='firwin')
        raw_task.filter(1, 40, fir_design='firwin')

        # Calculate PSD
        psd_rest = calculate_band_psd(raw_rest, raw_rest.info['sfreq'], freq_bands)
        psd_task = calculate_band_psd(raw_task, raw_task.info['sfreq'], freq_bands)

        # Flatten PSD values for machine learning input
        X_rest = np.concatenate([psd_rest[band] for band in freq_bands.keys()])
        X_task = np.concatenate([psd_task[band] for band in freq_bands.keys()])

        # Labels: 0 for rest, 1 for task
        y_rest = np.zeros(X_rest.shape[0])
        y_task = np.ones(X_task.shape[0])

        # Concatenate features and labels
        X_list.append(X_rest)
        X_list.append(X_task)
        y_list.append(y_rest)
        y_list.append(y_task)

    X = np.array(X_list)
    y = np.array(y_list)
    np.save('X_psd.npy', X)
    np.save('y_psd.npy', y)
    print(f"Saved PSD features to 'X_psd.npy' and labels to 'y_psd.npy'")

# Run the function to calculate and save PSD features
load_and_save_eeg_data(subject_info, freq_bands)


Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/Subject00_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 ...   181.998 secs...
Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/Subject00_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 H

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/Subject01_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 ...   181.998 secs...
Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/Subject01_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 H

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/Subject02_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 ...   181.998 secs...
Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/Subject02_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 H

...

Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/Subject34_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 ...   181.998 secs...
Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/Subject34_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 H

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/Subject35_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 ...   181.998 secs...
Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/Subject35_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 H

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Saved PSD features to 'X_psd.npy' and labels to 'y_psd.npy'


Preprocessing

## TSCeption

References:
https://torcheeg.readthedocs.io/en/v1.1.0/generated/torcheeg.models.TSCeption.html

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, classification_report

# Load saved PSD features and labels
X = np.load('X_psd.npy')
y = np.load('y_psd.npy')

# TSCeption expects 4D input: (samples, cnn_channel, EEG_channel, data_points)
X = X.reshape(X.shape[0], 1, X.shape[1], 1)
print(f"Reshaped X shape: {X.shape}")

# Ensuring y is a 1D array before one-hot encoding
if y.ndim > 1:
    y = y[:, 0]
print(f"Corrected y shape: {y.shape}")

# One-hot encode labels
y = to_categorical(y)
print(f"One-hot encoded y shape: {y.shape}")

# Flatten X for train_test_split (not used directly for training)
X_flat = X.reshape(X.shape[0], -1)
print(f"Flattened X shape: {X_flat.shape}")

# Split into train and test sets (stratify y)
X_train_flat, X_test_flat, y_train, y_test = train_test_split(
    X_flat, y, test_size=0.2, random_state=42, stratify=y
)

# Reshape back to original dimensions after the split for TSCeption
X_train = X_train_flat.reshape(X_train_flat.shape[0], 1, -1, 1)
X_test = X_test_flat.reshape(X_test_flat.shape[0], 1, -1, 1)

# Normalizing using StandardScaler
scaler = StandardScaler()

# Flatten the training data, fit the scaler, and reshape back to the original shape
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_train_scaled = scaler.fit_transform(X_train_flat)
X_train = X_train_scaled.reshape(X_train.shape)

# Flatten the testing data, apply the scaler, and reshape back to the original shape
X_test_flat = X_test.reshape(X_test.shape[0], -1)
X_test_scaled = scaler.transform(X_test_flat)
X_test = X_test_scaled.reshape(X_test.shape)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


Reshaped X shape: (72, 1, 105, 1)
Corrected y shape: (72,)
One-hot encoded y shape: (72, 2)
Flattened X shape: (72, 105)
X_train shape: (57, 1, 105, 1), y_train shape: (57, 2)
X_test shape: (15, 1, 105, 1), y_test shape: (15, 2)


In [ ]:
# Define TSCeption model
class TSception(nn.Module):
    def __init__(self, num_classes, input_size, sampling_rate=128, num_T=15, num_S=15, hidden=32, dropout_rate=0.5):
        super(TSception, self).__init__()

        self.num_classes = num_classes
        self.input_size = input_size
        self.sampling_rate = sampling_rate
        self.num_T = num_T
        self.num_S = num_S
        self.hidden = hidden
        self.dropout_rate = dropout_rate

        # Define layers
        self.conv1 = nn.Conv2d(1, num_T, kernel_size=(1, 1), stride=(1, 1))
        self.conv2 = nn.Conv2d(num_T, num_T * num_S, kernel_size=(1, 1), stride=(1, 1))
        self.conv3 = nn.Conv2d(num_T * num_S, num_S, kernel_size=(1, 1), stride=(1, 1))
        self.fc1 = nn.Linear(num_S * input_size[1], hidden)
        self.fc2 = nn.Linear(hidden, num_classes)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        # Input x shape: (batch_size, cnn_channel, EEG_channel, data_points)

        # Convolutional layers
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))

        # Flatten for fully connected layers
        x = x.view(x.size(0), -1)

        # Fully connected layers with dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

# Initialize TSception model
model = TSception(
    num_classes=2,  # Binary classification task, so 2
    input_size=(1, 105, 1),  # (cnn_channel, EEG_channel, data_points)
    sampling_rate=128,  # EEG data's sampling rate
    num_T=15,  # Number of temporal filters
    num_S=15,  # Size of hidden embedding
    hidden=32,  # Number of hidden units
    dropout_rate=0.3  # Dropout rate for regularization
)

print(model)


TSception(
  (conv1): Conv2d(1, 15, kernel_size=(1, 1), stride=(1, 1))
  (conv2): Conv2d(15, 225, kernel_size=(1, 1), stride=(1, 1))
  (conv3): Conv2d(225, 15, kernel_size=(1, 1), stride=(1, 1))
  (fc1): Linear(in_features=1575, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=2, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)


In [ ]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(np.argmax(y_train, axis=1), dtype=torch.long)
y_test_tensor = torch.tensor(np.argmax(y_test, axis=1), dtype=torch.long)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 30
batch_size = 16
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 5 == 0:
        model.eval()
        with torch.no_grad():
            outputs_test = model(X_test_tensor)
            _, predicted = torch.max(outputs_test, 1)
            accuracy = accuracy_score(y_test_tensor.numpy(), predicted.numpy())
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Test Accuracy: {accuracy:.4f}')


# Evaluate on test set and print metrics
model.eval()
with torch.no_grad():
    outputs_test = model(X_test_tensor)
    _, predicted = torch.max(outputs_test, 1)
    accuracy = accuracy_score(y_test_tensor.numpy(), predicted.numpy())
    print(f'Test Accuracy: {accuracy:.4f}')

    # Print classification report
    print(classification_report(y_test_tensor.numpy(), predicted.numpy()))

Epoch [5/30], Loss: 0.0111, Test Accuracy: 0.8000
Epoch [10/30], Loss: 0.0081, Test Accuracy: 0.8000
Epoch [15/30], Loss: 0.0083, Test Accuracy: 0.7333
Epoch [20/30], Loss: 0.0015, Test Accuracy: 0.7333
Epoch [25/30], Loss: 0.0028, Test Accuracy: 0.8000
Epoch [30/30], Loss: 0.0012, Test Accuracy: 0.8000
Test Accuracy: 0.8000
              precision    recall  f1-score   support

           0       0.83      0.71      0.77         7
           1       0.78      0.88      0.82         8

    accuracy                           0.80        15
   macro avg       0.81      0.79      0.80        15
weighted avg       0.80      0.80      0.80        15



## Inference:
Reduced Drop out rate to 0.3

30 epochs gives best accuracy here.

Achieved neat tradeoff between 80% Overall accuracy and decent recall for each class. In medical diagnostics where missing a positive case is critical, High Recall is preferred.